# Yet Another MLP Example with Theano

Adapted from: https://github.com/mbeissinger/intro_deep

In [30]:
from IPython.display import Image
import gzip
import pickle

In [31]:
# Download and unzip pickled version from here: http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = pickle.load(gzip.open('data/mnist.pkl.gz', 'rb'))
print "Shapes:"
print train_x.shape, train_y.shape
print valid_x.shape, valid_y.shape
print test_x.shape, test_y.shape

# print "--------------"
# print "Example input:"
# print train_x[0]
# print "Example label:"
# print train_y[0]

Shapes:
(50000L, 784L) (50000L,)
(10000L, 784L) (10000L,)
(10000L, 784L) (10000L,)


In [32]:
# Show example images - using tile_raster_images helper function from OpenDeep to get 28x28 image from 784 array.
from utils import tile_raster_images
from PIL import Image as pil_img

input_images = train_x[:25]
im = pil_img.fromarray(
    tile_raster_images(input_images, 
                       img_shape=(28, 28), 
                       tile_shape=(1, 25),
                       tile_spacing=(1, 1))
)
im.save("some_mnist_numbers.png")
Image(filename="some_mnist_numbers.png")


In [33]:
# Your basic Theano imports.
import theano
import theano.tensor as T

theano.config.floatX = 'float32'

x = T.matrix('x')


In [34]:
input_size = 28*28
hidden_nodes = 50
output_size = 10

In [35]:
# Compute the hidden layer from the input
import numpy
import numpy.random as rng

i = numpy.sqrt(6. / (input_size+hidden_nodes))
print i

0.084818892968


In [36]:
# W_x = numpy.asarray(rng.normal(loc=0.0, scale=.05, size=(input_size, hidden_nodes)), dtype=theano.config.floatX)
W_x = numpy.asarray(rng.uniform(low=-i, high=i, size=(input_size, hidden_nodes)), dtype=theano.config.floatX)
b_h = numpy.zeros(shape=(hidden_nodes,), dtype=theano.config.floatX)

In [37]:
b_h

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], dtype=float32)

In [38]:
W_x = theano.shared(W_x, name="W_x")
b_h = theano.shared(b_h, name="b_h")

h = T.tanh(
    T.dot(x, W_x) + b_h
)

In [39]:
# Compute the output class probabilities from the hidden layer
i = numpy.sqrt(6. / (hidden_nodes+output_size))
# W_h = numpy.asarray(rng.normal(loc=0.0, scale=.05, size=(hidden_nodes, output_size)), dtype=theano.config.floatX)

In [40]:
W_h = numpy.asarray(rng.uniform(low=-i, high=i, size=(hidden_nodes, output_size)), dtype=theano.config.floatX)
b_y = numpy.zeros(shape=(output_size,), dtype="float32")

In [41]:
W_h = theano.shared(W_h, name="W_h")
b_y = theano.shared(b_y, name="b_y")

y = T.nnet.softmax(
    T.dot(h, W_h) + b_y
)

In [42]:
# The actual predicted label
y_hat = T.argmax(y, axis=1)

In [43]:
# Find cost compared to correct labels
correct_labels = T.ivector("labels")

log_likelihood = T.log(y)[T.arange(correct_labels.shape[0]), correct_labels]
cost = -T.mean(log_likelihood)


In [44]:
# Compute gradient updates for the parameters
parameters = [W_x, b_h, W_h, b_y]
gradients = T.grad(cost, parameters)

learning_rate = 0.01
train_updates = [(param, param - learning_rate*gradient) for param, gradient in zip(parameters, gradients)]


In [45]:
# Compile function for training (changes parameters via updates) and testing (no updates)
f_train = theano.function(
    inputs=[x, correct_labels], 
    outputs=cost, 
    updates=train_updates, 
    allow_input_downcast=True
)

In [46]:
f_test = theano.function(
    inputs=[x], 
    outputs=y_hat, 
    allow_input_downcast=True
)

In [47]:
def run(batch_size=100, epochs=100, check_frequency=3):
    # Main training loop
    train_batches = len(train_x) / batch_size
    valid_batches = len(valid_x) / batch_size
    test_batches = len(test_x) / batch_size

    for epoch in range(epochs):
        print epoch+1, ":",

        train_costs = []
        train_accuracy = []
        for i in range(train_batches):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_labels = train_y[i*batch_size:(i+1)*batch_size]

            costs = f_train(batch_x, batch_labels)
            preds = f_test(batch_x)
            acc = sum(preds==batch_labels)/float(len(batch_labels))

            train_costs.append(costs)
            train_accuracy.append(acc)
        print "cost:", numpy.mean(train_costs), "\ttrain:", str(numpy.mean(train_accuracy)*100)+"%",

        valid_accuracy = []
        for i in range(valid_batches):
            batch_x = valid_x[i*batch_size:(i+1)*batch_size]
            batch_labels = valid_y[i*batch_size:(i+1)*batch_size]

            preds = f_test(batch_x)
            acc = sum(preds==batch_labels)/float(len(batch_labels))

            valid_accuracy.append(acc)
        print "\tvalid:", str(numpy.mean(valid_accuracy)*100)+"%",

        test_accuracy = []
        for i in range(test_batches):
            batch_x = test_x[i*batch_size:(i+1)*batch_size]
            batch_labels = test_y[i*batch_size:(i+1)*batch_size]

            preds = f_test(batch_x)
            acc = sum(preds==batch_labels)/float(len(batch_labels))

            test_accuracy.append(acc)
        print "\ttest:", str(numpy.mean(test_accuracy)*100)+"%"

        if (epoch+1) % check_frequency == 0:
            print 'saving filters...'
            weight_filters = pil_img.fromarray(
                    tile_raster_images(
                        W_x.get_value(borrow=True).T,
                        img_shape=(28, 28),
#                         tile_shape=(20, 25),
                        tile_shape=(7, 8),
                        tile_spacing=(1, 1)
                    )
                )
            weight_filters.save("mlp_filters_%d.png"%(epoch+1))

In [48]:
run(batch_size=100, epochs=200, check_frequency=3)

1 : cost: 1.10856 	train: 74.018% 	valid: 85.67% 	test: 84.98%
2 : cost: 0.599191 	train: 85.916% 	valid: 88.45% 	test: 87.97%
3 : cost: 0.484731 	train: 88.024% 	valid: 89.6% 	test: 89.19%
saving filters...
4 : cost: 0.428333 	train: 88.974% 	valid: 90.24% 	test: 89.85%
5 : cost: 0.393701 	train: 89.63% 	valid: 90.76% 	test: 90.24%
6 : cost: 0.369684 	train: 90.106% 	valid: 91.1% 	test: 90.75%
saving filters...
7 : cost: 0.351661 	train: 90.492% 	valid: 91.44% 	test: 91.01%
8 : cost: 0.337367 	train: 90.78% 	valid: 91.65% 	test: 91.23%
9 : cost: 0.325559 	train: 91.062% 	valid: 91.87% 	test: 91.46%
saving filters...
10 : cost: 0.315498 	train: 91.276% 	valid: 92.04% 	test: 91.62%
11 : cost: 0.306714 	train: 91.506% 	valid: 92.15% 	test: 91.74%
12 : cost: 0.298894 	train: 91.714% 	valid: 92.29% 	test: 91.89%
saving filters...
13 : cost: 0.291826 	train: 91.926% 	valid: 92.4% 	test: 92.1%
14 : cost: 0.285356 	train: 92.072% 	valid: 92.53% 	test: 92.3%
15 : cost: 0.279375 	train: 92.262%

118 : cost: 0.106336 	train: 97.142% 	valid: 96.43% 	test: 96.21%
119 : cost: 0.105748 	train: 97.152% 	valid: 96.43% 	test: 96.21%
120 : cost: 0.105167 	train: 97.168% 	valid: 96.43% 	test: 96.22%
saving filters...
121 : cost: 0.104591 	train: 97.184% 	valid: 96.42% 	test: 96.23%
122 : cost: 0.104022 	train: 97.19% 	valid: 96.44% 	test: 96.23%
123 : cost: 0.10346 	train: 97.214% 	valid: 96.44% 	test: 96.22%
saving filters...
124 : cost: 0.102903 	train: 97.24% 	valid: 96.44% 	test: 96.25%
125 : cost: 0.102352 	train: 97.258% 	valid: 96.49% 	test: 96.28%
126 : cost: 0.101807 	train: 97.276% 	valid: 96.5% 	test: 96.29%
saving filters...
127 : cost: 0.101268 	train: 97.286% 	valid: 96.5% 	test: 96.32%
128 : cost: 0.100735 	train: 97.302% 	valid: 96.52% 	test: 96.32%
129 : cost: 0.100207 	train: 97.326% 	valid: 96.53% 	test: 96.33%
saving filters...
130 : cost: 0.0996844 	train: 97.354% 	valid: 96.54% 	test: 96.34%
131 : cost: 0.0991674 	train: 97.372% 	valid: 96.53% 	test: 96.36%
132 : c